In [2]:
import os
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
import sys


In [2]:
cd /home/easha/FYP/hw2vec/examples

/home/easha/FYP/hw2vec/examples


In [3]:
# !python use_case_1.py


In [4]:
# import pickle
# import networkx as nx
# import matplotlib.pyplot as plt

# # Load the saved graph
# graph_path = "../hardware_graph.pkl"
# with open(graph_path, "rb") as f:
#     hardware_nxgraph = pickle.load(f)

# # Visualize the graph
# def visualize_graph(graph):
#     pos = nx.spring_layout(graph)
#     plt.figure(figsize=(12, 12))
#     nx.draw(graph, pos, with_labels=True, node_color='skyblue', node_size=2000, edge_color='gray', font_size=10, font_weight='bold')
#     plt.show()

# visualize_graph(hardware_nxgraph)

In [5]:
# !python use_case_2.py --yaml_path ./example_gnn4tj.yaml --raw_dataset_path ../assets/datasets/TJ-RTL-toy --data_pkl_path dfg_tj_rtl.pkl --model_path ../examples  --graph_type DFG --device cuda

In [6]:
# print(os.listdir(os.getcwd()))

# os.getcwd()

# os.path.dirname(os.getcwd())

# sys.path.append(os.path.dirname(os.getcwd()))

# sys.path
# os.path.dirname(sys.path[0])


In [7]:
import os, sys
# sys.path.append(os.path.dirname(sys.path[0]))
from hw2vec.hw2vec.config import Config
from hw2vec.hw2vec.hw2graph import *
from hw2vec.hw2vec.graph2vec.models import *
import torch
import numpy as np
import json
import yaml
from pathlib import Path
import torch.nn.functional as F


In [8]:
import torch

In [9]:
print(torch.__version__)

2.4.1+cu121


In [ ]:
from hw2vec.hw2vec.config import Config
from hw2vec.hw2vec.hw2graph import *
from hw2vec.hw2vec.graph2vec.models import *
import yaml
# Define a class to encapsulate the configuration dictionary
class Config:
    def __init__(self, config_dict):
        self.__dict__.update(config_dict)

# Configuration parameters
yaml_path = 'hw2vec/examples/example_gnn4tj.yaml'
# raw_dataset_path = Path('../assets/testing')
# raw_dataset_path = Path('../../AES-T100/src/TjIn') 
# raw_dataset_path = Path('../../RS232-T100/src/') 
# raw_dataset_path = Path('../../PIC16F84-T200/src/TjIn') 
# raw_dataset_path = Path('../../b15-T100/src/TjIn') 
# raw_dataset_path = Path('../../s15850-T100/src/TjIn') 
# raw_dataset_path = Path('../../ISCAS85/test2') 

raw_dataset_path = Path('/home/easha/FYP/iscas/ISCAS89/test3') 

# verilog_file_path = Path('../../test2') 
verilog_file_path = raw_dataset_path

# raw_dataset_path = Path('../../VerilogTest') 

data_pkl_path = Path('testing.pkl')
graph_type = 'DFG'
device = 'cuda'

with open(yaml_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update config_dict with additional parameters
config_dict.update({
    'raw_dataset_path': str(raw_dataset_path),
    'data_pkl_path': str(data_pkl_path),
    'graph_type': graph_type,
    'device': device
})

# Initialize the configuration class
config = Config(config_dict)

# verilog_file_path = Path('../assets/testing') 

# verilog_file_path = Path('../../AES-T100/src/TjIn')
# verilog_file_path = Path('../../RS232-T100/src/')  
# verilog_file_path = Path('../../PIC16F84-T200/src/TjIn')  
# verilog_file_path = Path('../../b15-T100/src/TjIn')  
# verilog_file_path = Path('../../s15850-T100/src/TjIn')  
# verilog_file_path = Path('../../ISCAS85/test2')  

# verilog_file_path = Path('../../S-seriesprocessed/S-series/TjIn/s15850-T100') 

# verilog_file_path = Path('../../VerilogTest') 

nx_graphs = []
hw2graph = HW2GRAPH(config)

# Process the single Verilog file
hw_graph = hw2graph.code2graph(verilog_file_path)
nx_graphs.append(hw_graph)

data_proc = DataProcessor(config)
for hw_graph in nx_graphs:
    data_proc.process(hw_graph)
data_proc.cache_graph_data(config.data_pkl_path)
data=data_proc.get_graphs()

# model_path = "model_trained.pth"
model_path = "/home/easha/FYP/hw2vec/assets/pretrained_DFG_TJ_RTL/model.pth"

# config_path =  "model_trained.cfg"
config_path = "/home/easha/FYP/hw2vec/assets/pretrained_DFG_TJ_RTL/model.cfg"

model = GRAPH2VEC(config)
model_path = Path(model_path)
if model_path.exists():
    model.load_model(config_path, model_path)

model.to(config.device)

def inference(model, data_loader, config):
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(data_loader):
            data = data.to(config.device)  # Move data to the correct device

            output, attn = model.embed_graph(data.x, data.edge_index, data.batch)
            output = model.mlp(output)
            output = F.log_softmax(output, dim=1)
            # print(output)
            


        # outputs = torch.cat(output).reshape(-1, 2).detach()

        

        # labels_tensor = torch.LongTensor(labels).detach()
        outputs_tensor = torch.FloatTensor(output.cpu()).detach()
        preds = outputs_tensor.max(1)[1].detach()

    return outputs_tensor, preds

train_loader = DataLoader(data, shuffle=True, batch_size=1)

outputs_tensor, preds=inference(model, train_loader, config)

print(f"The output tensor is: {outputs_tensor}")
print(f"The predicted class is: {'TJ Free' if preds[0] == 0 else 'TJ Detected'}")

/home/easha/FYP/iscas/ISCAS89/test3/topModule.v , 51282 , 63450 , 22.4622962474823


FileNotFoundError: [Errno 2] No such file or directory: 'model_trained.pth'

In [11]:
import torch
torch.cuda.is_available()

True

In [ ]:
# import os, sys
# # sys.path.append(os.path.dirname(sys.path[0]))
# from hw2vec.hw2vec.config import Config
# from hw2vec.hw2vec.hw2graph import *
# from hw2vec.hw2vec.graph2vec.models import *
# import torch
# import numpy as np
# import json
# import yaml
# from pathlib import Path
# import torch.nn.functional as F
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd

# # Define a class to encapsulate the configuration dictionary
# class Config:
#     def __init__(self, config_dict):
#         self.__dict__.update(config_dict)

# # Configuration parameters
# yaml_path = 'yaml_files/example_gnn4tj.yaml'
# # raw_dataset_path = Path('../assets/testing')
# # raw_dataset_path = Path('../../AES-T100/src/TjIn')
# # raw_dataset_path = Path('../../RS232-T100/src/')
# # raw_dataset_path = Path('../../PIC16F84-T200/src/TjIn')
# # raw_dataset_path = Path('../../b15-T100/src/TjIn')
# # raw_dataset_path = Path('../../s15850-T100/src/TjIn')
# # raw_dataset_path = Path('../../ISCAS85/test2')

# # tj_free_data_dir_path = Path('../../S-seriesprocessed/S-series/TjFree')
# # base_data_path = Path('../../TJ-RTL-toy-rand-2-test')
# base_data_path = Path('/home/easha/FYP/S-seriesfinal/S-series')


# tj_free_data_dir_path = os.path.join(base_data_path, "TjFree")
# error_files  = []
# total_tj_free_files = 0
# total_tj_detected_in_tj_free_files = 0
# for folder in os.listdir(tj_free_data_dir_path):
#     # check if folder is a directory
#     if not os.path.isdir(os.path.join(tj_free_data_dir_path, folder)):
#         continue

#     print(f"Processing TJFree {folder}  ....")

#     raw_dataset_path = Path(os.path.join(tj_free_data_dir_path, folder))
#     # raw_dataset_path = Path('../../S-seriesprocessed/S-series/TjFree/s15850-T100')

#     # raw_dataset_path = Path('../../VerilogTest')

#     data_pkl_path = Path(f'temp_{folder}.pkl')
#     # data_pkl_path = Path('data_pkl_path.pkl')

#     graph_type = 'DFG'
#     device = 'cuda'

#     with open(yaml_path, 'r') as f:
#         config_dict = yaml.safe_load(f)

#     # Update config_dict with additional parameters
#     config_dict.update({
#         'raw_dataset_path': str(raw_dataset_path),
#         'data_pkl_path': str(data_pkl_path),
#         'graph_type': graph_type,
#         'device': device
#     })

#     # Initialize the configuration class
#     config = Config(config_dict)

#     # verilog_file_path = Path('../assets/testing')

#     # verilog_file_path = Path('../../AES-T100/src/TjIn')
#     # verilog_file_path = Path('../../RS232-T100/src/')
#     # verilog_file_path = Path('../../PIC16F84-T200/src/TjIn')
#     # verilog_file_path = Path('../../b15-T100/src/TjIn')
#     # verilog_file_path = Path('../../s15850-T100/src/TjIn')
#     # verilog_file_path = Path('../../ISCAS85/test2')

#     verilog_file_path = raw_dataset_path
#     # verilog_file_path = Path('../../S-seriesprocessed/S-series/TjFree/s15850-T100')


#     # verilog_file_path = Path('../../VerilogTest')

#     nx_graphs = []
#     hw2graph = HW2GRAPH(config)
#     data_proc = DataProcessor(config)

#     if data_pkl_path.exists():
#         data_proc.read_graph_data_from_cache(data_pkl_path)
#         print("using cache")
#     else:
#         # Process the single Verilog file
#         try:
#             hw_graph = hw2graph.code2graph(verilog_file_path)
#         except Exception as e:
#             print(f"Error in processing {folder}. \n Error:  {e}")
#             error_files.append({
#                 "name": folder,
#                 "error": e
#             })
#             continue
#         nx_graphs.append(hw_graph)

#         for hw_graph in nx_graphs:
#             data_proc.process(hw_graph)
#         data_proc.cache_graph_data(config.data_pkl_path)
#     data=data_proc.get_graphs()

#     model_path = "hw2vec/assets/pretrained_DFG_TJ_RTL/model.pth"
#     config_path = "hw2vec/assets/pretrained_DFG_TJ_RTL/model.cfg"


#     model = GRAPH2VEC(config)
#     model_path = Path(model_path)
#     if model_path.exists():
#         model.load_model(config_path, model_path)
#     else:
#         raise Exception("Model Does not exist")

#     model.to(config.device)

#     def inference(model, data_loader, config):
#         with torch.no_grad():
#             model.eval()
#             for i, data in enumerate(data_loader):
#                 data = data.to(config.device)  # Move data to the correct device

#                 output, attn = model.embed_graph(data.x, data.edge_index, data.batch)
#                 output = model.mlp(output)
#                 output = F.log_softmax(output, dim=1)
#                 # print(output)



#             # outputs = torch.cat(output).reshape(-1, 2).detach()



#             # labels_tensor = torch.LongTensor(labels).detach()
#             outputs_tensor = torch.FloatTensor(output.cpu()).detach()
#             preds = outputs_tensor.max(1)[1].detach()

#         return outputs_tensor, preds

#     train_loader = DataLoader(data, shuffle=True, batch_size=1)

#     outputs_tensor, preds=inference(model, train_loader, config)

#     # print(f"The output tensor is: {outputs_tensor}")

#     trojan_detected = preds[0] == 1
#     print(f"The predicted class is: {'TJ Free' if preds[0] == 0 else 'TJ Detected'}")

#     total_tj_free_files += 1
#     if trojan_detected:
#         total_tj_detected_in_tj_free_files += 1

# print(error_files)

# # Now we will process the files with trojans
# tj_data_dir_path = os.path.join(base_data_path, "TjIn")
# # tj_data_dir_path = Path('../../S-seriesprocessed/S-series/TjIn')
# # tj_data_dir_path = Path('../assets/datasets/TJ-RTL-tsoy/TjIn')

# error_files = []

# total_tj_files = 0
# total_tj_detected_in_tj_files = 0
# for folder in os.listdir(tj_data_dir_path):
#     # check if folder is a directory
#     if not os.path.isdir(os.path.join(tj_data_dir_path, folder)):
#         print("skipping file...")
#         continue

#     print(f"Processing TJIn {folder}  ....")


#     raw_dataset_path = Path(os.path.join(tj_data_dir_path, folder))
#     # raw_dataset_path = Path('../../S-seriesprocessed/S-series/TjFree/s15850-T100')

#     # raw_dataset_path = Path('../../VerilogTest')

#     data_pkl_path = Path(f'temp_{folder}.pkl')
#     graph_type = 'DFG'
#     device = 'cuda'

#     with open(yaml_path, 'r') as f:
#         config_dict = yaml.safe_load(f)

#     # Update config_dict with additional parameters
#     config_dict.update({
#         'raw_dataset_path': str(raw_dataset_path),
#         'data_pkl_path': str(data_pkl_path),
#         'graph_type': graph_type,
#         'device': device
#     })

#     # Initialize the configuration class
#     config = Config(config_dict)

#     # verilog_file_path = Path('../assets/testing')

#     # verilog_file_path = Path('../../AES-T100/src/TjIn')
#     # verilog_file_path = Path('../../RS232-T100/src/')
#     # verilog_file_path = Path('../../PIC16F84-T200/src/TjIn')
#     # verilog_file_path = Path('../../b15-T100/src/TjIn')
#     # verilog_file_path = Path('../../s15850-T100/src/TjIn')
#     # verilog_file_path = Path('../../ISCAS85/test2')

#     verilog_file_path = raw_dataset_path

#     # verilog_file_path = Path('../../S-seriesprocessed/S-series/TjFree/s15850-T100')

#     # verilog_file_path = Path('../../VerilogTest')

#     nx_graphs = []
#     hw2graph = HW2GRAPH(config)
#     data_proc = DataProcessor(config)

#     if data_pkl_path.exists():
#         data_proc.read_graph_data_from_cache(data_pkl_path)
#         print("using cache")
#     else:
#         # Process the single Verilog file
#         try:
#             hw_graph = hw2graph.code2graph(verilog_file_path)
#         except Exception as e:
#             print(f"Error in processing {folder}. \n Error:  {e}")
#             error_files.append({
#                 "name": folder,
#                 "error": e
#             })
#             continue
#         nx_graphs.append(hw_graph)

#         for hw_graph in nx_graphs:
#             data_proc.process(hw_graph)
#         data_proc.cache_graph_data(config.data_pkl_path)
#     data=data_proc.get_graphs()

   
#     model_path = "hw2vec/assets/pretrained_DFG_TJ_RTL/model.pth"
#     config_path = "hw2vec/assets/pretrained_DFG_TJ_RTL/model.cfg"


#     model = GRAPH2VEC(config)
#     model_path = Path(model_path)
#     if model_path.exists():
#         model.load_model(config_path, model_path)
#     else:
#         raise Exception("Model Does not exist")
#     model.to(config.device)

#     def inference(model, data_loader, config):
#         with torch.no_grad():
#             model.eval()
#             for i, data in enumerate(data_loader):
#                 data = data.to(config.device)  # Move data to the correct device

#                 output, attn = model.embed_graph(data.x, data.edge_index, data.batch)
#                 output = model.mlp(output)
#                 output = F.log_softmax(output, dim=1)
#                 # print(output)



#             # outputs = torch.cat(output).reshape(-1, 2).detach()



#             # labels_tensor = torch.LongTensor(labels).detach()
#             outputs_tensor = torch.FloatTensor(output.cpu()).detach()
#             preds = outputs_tensor.max(1)[1].detach()

#         return outputs_tensor, preds

#     train_loader = DataLoader(data, shuffle=True, batch_size=1)

#     outputs_tensor, preds=inference(model, train_loader, config)

#     # print(f"The output tensor is: {outputs_tensor}")

#     trojan_detected = preds[0] == 1
#     print(f"The predicted class is: {'TJ Free' if preds[0] == 0 else 'TJ Detected'}")

#     total_tj_files += 1
#     if trojan_detected:
#         total_tj_detected_in_tj_files += 1
# print(error_files)

# # compute the accuracy and confusion matrix
# #

# print(f"Total TJ Free files: {total_tj_free_files}")
# print(f"Total TJ Detected in TJ Free files (False Positives): {total_tj_detected_in_tj_free_files}")
# print(f"Total TJ Free accuracy: {(total_tj_free_files - total_tj_detected_in_tj_free_files) / total_tj_free_files}")

# print(f"Total TJ files: {total_tj_files}")
# print(f"Total TJ Detected in TJ files (True Positives): {total_tj_detected_in_tj_files}")
# print(f"Total TJ accuracy: {total_tj_detected_in_tj_files / total_tj_files}")

# # confusion matrix

# # True Positives
# # False Positives
# # True Negatives
# # False Negatives

# # Confusion matrix components
# TP = total_tj_detected_in_tj_files      # True Positive: correctly detected trojan-inserted
# FP = total_tj_detected_in_tj_free_files # False Positive: wrongly detected trojan in trojan-free
# FN = total_tj_files - TP                # False Negative: missed detection of trojan
# TN = total_tj_free_files - FP           # True Negative: correctly identified trojan-free

# # Print metrics
# print(f"True Positives (TjIn correctly detected): {TP}")
# print(f"False Positives (TjFree wrongly flagged): {FP}")
# print(f"True Negatives (TjFree correctly identified): {TN}")
# print(f"False Negatives (TjIn missed): {FN}")

# # Evaluation metrics
# eps = 1e-8  # To avoid division by zero
# Accuracy = (TP + TN) / (TP + TN + FP + FN + eps)
# Precision = TP / (TP + FP + eps)
# Recall = TP / (TP + FN + eps)
# F1 = 2 * (Precision * Recall) / (Precision + Recall + eps)

# print(f"Accuracy: {Accuracy:.4f}")
# print(f"Precision: {Precision:.4f}")
# print(f"Recall: {Recall:.4f}")
# print(f"F1 Score: {F1:.4f}")

# # Create confusion matrix as a DataFrame
# # Index = Ground Truth, Columns = Predicted
# conf_matrix = {
#     'TjFree': [TN, FP],  # Predicted as TjFree or TjIn
#     'TjIn': [FN, TP]
# }

# df = pd.DataFrame(conf_matrix, index=['TjFree', 'TjIn'])  # Ground Truth: TjFree (negative), TjIn (positive)

# # Plotting
# sns.heatmap(
#     df,
#     annot=True,
#     fmt='d',
#     cmap='Blues',
#     xticklabels=['Pred: TjFree (Negative)', 'Pred: TjIn (Positive)'],
#     yticklabels=['GT: TjFree (Negative)', 'GT: TjIn (Positive)']
# )
# plt.xlabel('Predicted Label')
# plt.ylabel('Ground Truth Label')
# plt.title('Confusion Matrix')
# plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/easha/FYP/S-seriesfinal/S-series/TjFree'